In [1]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Input, Concatenate, BatchNormalization, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

# Descargar la lista de stopwords si no está ya descargada
nltk.download('stopwords')

# Cargar las stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingfeng.chen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = Loader.load_train()

c:\Users\lingfeng.chen\Downloads\Cars_Price_Prediction-main\Cars_Price_Prediction-main\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingfeng.chen\Downloads\Cars_Price_Prediction-main\Cars_Price_Prediction-main\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingfeng.chen\Downloads\Cars_Price_Prediction-main\Cars_Price_Prediction-main\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingfeng.chen\Downloads\Cars_Price_Prediction-main\Cars_Price_Prediction-main\NLP\..\utils\loader.py:145: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior w

In [3]:
descriptions = [col for col in train.columns if "description" in col]

print(descriptions)

def custom_concat(row, cols):
    # Construir la descripción con lógica condicional basada en el valor de la celda
    parts = []
    for col_name in cols:  # Cambio para iterar solo sobre las columnas especificadas
        if col_name in row.index:  # Verificar que el nombre de la columna esté en el DataFrame
            value = row[col_name]
            if value == "no tiene" or not isinstance(value, str):
                parts.append(f"no tiene {col_name}")
            else:
                parts.append(str(value))  # Convertir a string para evitar problemas al unir
    # Unir todas las partes con espacios
    return ' '.join(parts)

# Aplicar la función al DataFrame
train['full_description'] = train.apply(custom_concat, axis=1, args=(descriptions,))
#train[descriptions].apply(lambda col: col.apply(lambda x: type(x).__name__).value_counts())

#descriptions.append("idx")
#descriptions.append("price")
#descriptions.append("km")

['motor_description', 'transmission_description', 'parking_sensors_description', 'Exterior_Llantas_description', 'Exterior_Molduras_description', 'Exterior_Neumáticos_description', 'Exterior_Puerta_description', 'Exterior_Retrovisor_description', 'Confort_Volante_description', 'seat_description', 'head_supporter_description']


In [4]:
filtered_columns = ["price", "km", "fuelType", "full_description"]
train = train[filtered_columns]
train.dropna(inplace=True)
train

,price,km,fuelType,full_description
idx,,,,
105953,-0.077067,-0.219436,Gasolina,"Motor de 1,3 litros ( 1.332 cc ) , cuatro cili..."
15683,-0.193101,0.362457,Diésel,"Motor de 2,0 litros ( 1.969 cc ) , cuatro cili..."
153684,-0.308670,0.531285,Diésel,"Motor de 1,6 litros ( 1.560 cc ) , cuatro cili..."
156928,-0.058037,-0.439005,Híbrido,"Motor de 1,8 litros ( 1.798 cc ) , cuatro cili..."
24443,-0.935252,2.206234,Diésel,"Motor de 1,9 litros ( 1.910 cc ) , 4 cilindros..."
...,...,...,...,...
19845,-0.058037,-0.113179,Diésel,"Motor de 1,6 litros ( 1.598 cc ) , cuatro cili..."
102652,-0.889071,0.848626,Gasolina,"Motor de 2,0 litros ( 1.997 cc ) , 4 cilindros..."
101658,-0.309134,0.509224,Gas natural (CNG),"Motor de 1,4 litros ( 1.395 cc ) , cuatro cili..."


In [5]:
#Parameters
verb_size = 200

In [6]:
def preprocess_text(text):
    # Convertir a minúsculas y eliminar caracteres especiales (pero mantener los números)
    text = text.lower()
    text = re.sub(r'[^\w\s\d]', '', text)  # Eliminar signos de puntuación, pero mantener números
    tokens = text.split()
    
    # Filtrar tokens, eliminando las stopwords
    filtered_tokens = [token for token in tokens if token not in spanish_stopwords]
    
    return filtered_tokens

train['tokens'] = train['full_description'].apply(preprocess_text)
model_w2v = Word2Vec(sentences=train['tokens'], vector_size=verb_size, window=1, min_count=3, workers=8)
word_vectors = model_w2v.wv

def get_average_embedding(tokens, model):
    embeddings = [model[word] for word in tokens if word in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size) 
train['embedding'] = train['tokens'].apply(lambda x: get_average_embedding(x, word_vectors))

In [7]:
train['price']

idx
105953   -0.077067
15683    -0.193101
153684   -0.308670
156928   -0.058037
24443    -0.935252
            ...   
19845    -0.058037
102652   -0.889071
101658   -0.309134
69350    -0.199598
13515     1.663857
Name: price, Length: 115597, dtype: float64

In [8]:
# Load data (assuming df is your DataFrame containing the required columns)
texts = train['full_description'].values  # descriptions
embeddings = train['embedding'].values  # embeddings

# Tokenization and sequence padding
tokenizer = Tokenizer(num_words=verb_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences,  maxlen=verb_size, padding='post')

# Split data into training and testing sets
train_data, test_data, train_embeddings, test_embeddings, train_prices, test_prices = train_test_split(
    data, embeddings, train['price'], test_size=0.2, random_state=42, stratify=train['fuelType'])

# Normalize embeddings
train_embeddings = tf.keras.utils.normalize(train_embeddings)
test_embeddings = tf.keras.utils.normalize(test_embeddings)
train_embeddings = np.transpose(train_embeddings)
test_embeddings = np.transpose(test_embeddings)

In [9]:
test_prices

idx
158363    0.150823
157347   -0.020907
101674   -0.211666
92113    -0.782552
57219    -0.573227
            ...   
63923    -0.030189
88274    -0.216307
22033     0.122975
55716    -0.796476
120916    0.638165
Name: price, Length: 23120, dtype: float64

In [10]:
def process_embedding(embed):
    standard_length = 200
    flat_embed = np.concatenate([np.array(e).flatten() for e in embed])  # Flatten
    # Pad with zeros or truncate to the standard length
    if len(flat_embed) > standard_length:
        return flat_embed[:standard_length]
    elif len(flat_embed) < standard_length:
        return np.pad(flat_embed, (0, standard_length - len(flat_embed)), mode='constant')
    return flat_embed

train_embeddings = np.array([process_embedding(e) for e in train_embeddings], dtype=np.float32)
train_embeddings = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)

test_embeddings = np.array([process_embedding(e) for e in test_embeddings], dtype=np.float32)
test_embeddings = tf.convert_to_tensor(test_embeddings, dtype=tf.float32)

train_data = tf.convert_to_tensor(train_data, dtype=tf.int32)
train_prices = tf.convert_to_tensor(train_prices, dtype=tf.float32)

test_data = tf.convert_to_tensor(test_data, dtype=tf.int32)
test_prices = tf.convert_to_tensor(test_prices, dtype=tf.float32)

In [11]:
test_prices

<tf.Tensor: shape=(23120,), dtype=float32, numpy=
array([ 0.15082344, -0.02090651, -0.21166599, ...,  0.12297534,
       -0.79647607,  0.6381652 ], dtype=float32)>

In [12]:
# Text input branch
text_input = Input(shape=(200,), dtype='int32')
text_embed_layer = Embedding(input_dim=verb_size, output_dim=verb_size, input_length=train_data.shape[1])
text_embed = text_embed_layer(text_input)
lstm_out = LSTM(128)(text_embed)

c:\Users\lingfeng.chen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# Embedding input branch
embed_input = Input(shape=(train_embeddings.shape[1],), dtype = 'int32')
dense_embed = Dense(128, activation='relu')(embed_input)

# Concatenate both branches
concat = Concatenate()([lstm_out, dense_embed])
hidden = Dense(256, activation='relu')(concat)
output = Dense(1, activation='linear')(hidden)  # Output layer for price prediction

model = Model(inputs=[text_input, embed_input], outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 200, 200)  │     40,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │    168,448 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     25,728 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        257 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 300,225 (1.15 MB)

 Trainable params: 300,225 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
print(train_data.shape)      # Should be (number of samples, sequence length)
print(train_embeddings.shape)

(92477, 200)
(92477, 200)


In [15]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

In [16]:
# Train the model
model.fit([train_data, train_embeddings], train_prices,
          validation_data=([test_data, test_embeddings], test_prices),
          epochs=1, batch_size=64)

c:\Users\lingfeng.chen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_3']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1445/1445 ━━━━━━━━━━━━━━━━━━━━ 230s 158ms/step - loss: 0.7471 - mean_squared_error: 0.7471 - val_loss: 0.4432 - val_mean_squared_error: 0.4432


In [17]:
test_loss, test_mse = model.evaluate([test_data, test_embeddings], test_prices)
print(f'Test MSE: {test_mse}')

723/723 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - loss: 0.4170 - mean_squared_error: 0.4170
Test MSE: 0.44321566820144653


In [18]:
predicted_prices = model.predict([test_data, test_embeddings])

# Create a DataFrame for actual vs predicted prices
import pandas as pd

# Assuming test_prices is a 1D array, we can convert it to a DataFrame
results_df = pd.DataFrame({
    'Actual Price': test_prices,
    'Predicted Price': predicted_prices.flatten()  # Flatten in case it's a 2D array
})

# Display the first few rows of the DataFrame
print(results_df.head())

723/723 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step
   Actual Price  Predicted Price
0      0.150823         0.060198
1     -0.020907         0.041577
2     -0.211666        -0.216664
3     -0.782552        -0.403310
4     -0.573227        -0.615146
